In [3]:
import numpy as np
import math
import scipy.stats as stats
from scipy.optimize import minimize
import pandas as pd
import statsmodels.api as sm

# Portfolio VaR

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.decomposition import PCA

# Reading the portfolio data into a DataFrame
portfolio = pd.read_csv("portfolio.csv")
returns = pd.read_csv('DailyReturn.csv')
prices = pd.read_csv('DailyPrices.csv')
portfolio = portfolio[~portfolio['Stock'].isin(['ELV', 'MMC', 'VRTX', 'REGN', 'CB', 'CI', 'ETN', 'SLB', 'PGR', 'BSX'])]
portfolio

,Portfolio,Stock,Holding
0,A,AAPL,158
1,A,MSFT,178
2,A,AMZN,110
3,A,NVDA,54
4,A,GOOGL,69
...,...,...,...
94,C,SCHW,158
95,C,LRCX,194
96,C,ZTS,121
97,C,C,83


### Covariance Matrix Function

In [11]:
def exp_weighted_cov(returns, lambda_=0.97):
    returns = returns.values
    mean_return = np.mean(returns, axis=0)
    normalized_returns = returns - mean_return

    n_timesteps = normalized_returns.shape[0]
    cov = np.cov(returns, rowvar=False)

    for t in range(1, n_timesteps):
        cov = lambda_ * cov + (1 - lambda_) * np.outer(normalized_returns[t], normalized_returns[t])
    return cov

In [12]:
def return_calculate(prices, method="DISCRETE", dateColumn="Date"):
    vars_ = prices.columns
    nVars = len(vars_)
    vars_ = [var for var in vars_ if var != dateColumn]
    if nVars == len(vars_):
        raise ValueError(f"dateColumn: {dateColumn} not in DataFrame: {vars_}")
    nVars = nVars - 1
    p = prices[vars_].to_numpy()
    n, m = p.shape
    p2 = np.empty((n-1, m))
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\",\"DISCRETE\")")
    dates = prices[dateColumn].iloc[1:n].to_numpy()
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars_[i]] = p2[:, i]
    return out


In [62]:
# Define a function to get the current price of a portfolio, option to calculate delta
def get_portfolio_price(portfolio, prices, portfolio_name, Delta=False):
    # If portfolio_name is "All", then aggregate holdings by stock across all portfolios
    if portfolio_name == "All":
        # Drop the 'Portfolio' column as we're aggregating across all portfolios
        assets = portfolio.drop('Portfolio', axis=1)
        # Group by 'Stock' and sum up the 'Holding' values, resetting index to have 'Stock' as a column
        assets = assets.groupby(["Stock"], as_index=False)["Holding"].sum()
    else:
        # If a specific portfolio is named, just select that portfolio's stocks and holdings
        assets = portfolio[portfolio["Portfolio"] == portfolio_name]     
    # Extract the list of stock codes from the assets DataFrame
    stock_codes = list(assets["Stock"])
    # Concatenate the 'Date' column from prices with the selected stock columns, aligning data by date
    assets_prices = pd.concat([prices["Date"], prices[stock_codes]], axis=1) 
    # Calculate the current portfolio price by taking the dot product of the latest prices and holdings
    current_price = np.dot(prices[assets["Stock"]].tail(1), assets["Holding"])
    # Extract the holdings for the assets
    holdings = assets["Holding"]   
    # If Delta is True, calculate the delta values
    if Delta == True:
        # Multiply each holding by its corresponding latest stock price to get the asset value
        asset_values = assets["Holding"].values.reshape(-1, 1) * prices[assets["Stock"]].tail(1).T.values
        # Calculate delta by dividing the asset values by the total current price of the portfolio
        delta = asset_values / current_price       
        # Return the current price of the portfolio, asset prices with dates, and delta values
        return current_price, assets_prices, delta   
    # If Delta is False, just return the current price, asset prices with dates, and holdings
    return current_price, assets_prices, holdings

### 

# Delta-Normal VaR

In [63]:
# Calculate with Delta Normal
def calculate_delta_var(portfolio, prices, alpha=0.05, lambda_=0.94, portfolio_name="All"):
    # 计算投资组合的当前价格和资产价格
    current_price, assets_prices, delta = get_portfolio_price(portfolio, prices, portfolio_name, Delta=True)
    # 计算资产的收益率
    returns = return_calculate(assets_prices, dateColumn="Date").drop('Date', axis=1)
    # 使用指数加权协方差矩阵
    assets_cov = ewCovar(returns, lambda_)
    # 计算VaR的delta值
    p_sig = np.sqrt(np.transpose(delta) @ assets_cov @ delta)
    # 根据delta-normal方法计算VaR
    var_delta = -current_price * stats.norm.ppf(alpha) * p_sig
    return current_price[0], var_delta[0][0]

# Historical VaR

In [86]:
# Calculate with historical simulation
def calculate_historic_var(portfolio, prices, alpha=0.05, n_simulation=1000, portfolio_name="All"):
    # 与delta-normal方法类似，先获取当前价格和资产价格
    current_price, assets_prices,delta = get_portfolio_price(portfolio, prices, portfolio_name)
    # 计算资产收益率
    returns = return_calculate(assets_prices, dateColumn="Date").drop("Date", axis=1)
    # 从资产收益率中抽样，构建模拟的收益率矩阵
    sim_returns = returns.sample(n_simulation, replace=True)
    # 计算模拟的投资组合价格
    assets_prices = assets_prices.iloc[:,1:]
    #simulated portfolio prices
    sim_prices = np.dot((sim_returns+1), assets_prices.tail(1).values.reshape(assets_prices.shape[1],))
    # 计算历史模拟VaR值
    var_hist = -np.percentile(sim_prices, alpha*100)
    return current_price[0], var_hist, sim_prices

# Results

In [87]:
# Print the results
current_price, delta_var = calculate_delta_var(portfolio, prices, portfolio_name='A')
current_price, hist_var, hist_sim_prices = calculate_historic_var(portfolio, daily_prices, portfolio_name='A')

print("The current value for A is: {:.2f}".format(current_price))
print("VaR for A using Delta Normal is: {:.2f}".format(delta_var))
print("VaR for A using Historic Simulation is: {:.2f}\n".format(hist_var))

The current value for A is: 1089316.16
VaR for A using Delta Normal is: 15206.39
VaR for A using Historic Simulation is: -8728.63

